In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ExpSineSquared, RBF, WhiteKernel
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
# 1. LOAD THE CSV AND PREPARE TIMESTAMP INDEX
df = pd.read_csv(
    './data/renewables/pv_with_weather_data.csv',
    comment='#',
    parse_dates=['time'],
    index_col='time'
)
df = df.asfreq('H')  # ensure a regular hourly index

# Keep only the PV ("electricity") column as our target
df = df[['electricity']].rename(columns={'electricity': 'pv'})

/var/folders/1m/qt1qgmf14_gb0hy_4g7323hc0000gn/T/ipykernel_42030/4043715517.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.asfreq('H')  # ensure a regular hourly index


In [4]:
# 2. DEFINE TRAIN/TEST SPLIT (last calendar year 2024 is test)
train_end = pd.Timestamp('2023-12-31 23:00:00')
test_start = train_end + pd.Timedelta(hours=1)

df_train = df.loc[:train_end]
df_test  = df.loc[test_start:]

In [5]:
# 3. ENCODE "TIME" AS A 1D NUMERIC ARRAY
#    We will simply assign an integer t_i for each hour in chronological order.
#    This is what the GP will use as its "X" input.
t_all = np.arange(len(df))                # 0,1,2,... for every hour in full dataset
t_train = t_all[:len(df_train)].reshape(-1, 1)
t_test  = t_all[len(df_train): ].reshape(-1, 1)

y_train = df_train['pv'].values
y_test  = df_test['pv'].values

In [6]:
# 4. CHOOSE A KERNEL: (ExpSineSquared × RBF) + WhiteKernel
#    - ExpSineSquared for strict 24h periodicity
#    - RBF to allow the daily‐cycle amplitude/phase to drift slowly
#    - WhiteKernel to absorb observation noise
k_periodic = ExpSineSquared(length_scale=1.0, periodicity=24.0)
k_rbf      = RBF(length_scale=24.0 * 30.0)       # one‐month‐scale drift
k_noise    = WhiteKernel(noise_level=1e-2)       

kernel = (k_periodic * k_rbf) + k_noise

In [7]:
# 5. INSTANTIATE & FIT THE GP REGRESSOR
gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,             # zero‐center & scale the target
    n_restarts_optimizer=10,      # try multiple initializations
    random_state=0
)
gp.fit(t_train, y_train)

: 

In [ ]:
# 6. FORECASTING: MAKE PREDICTIONS ON df_test
#    We want to extract forecasts (mean + uncertainty) for specific dates:
target_dates = ['2024-01-01', '2024-01-08', '2024-01-31']

# Pre‐allocate a DataFrame to hold MAE/RMSE results
results = []

for date_str in target_dates:
    # a) Build a 24‐hour range for that date, at hourly resolution
    day_start = pd.Timestamp(date_str + ' 00:00:00')
    day_end   = pd.Timestamp(date_str + ' 23:00:00')
    idx_range = pd.date_range(day_start, day_end, freq='H')

    # b) Convert those timestamps to the same integer‐hour index as t_all
    #    We can look them up by .get_loc on df.index
    try:
        positions = [df.index.get_loc(ts) for ts in idx_range]
    except KeyError:
        # If any of the 24h are missing, skip that day
        print(f"  → {date_str} missing from index, skipping.")
        continue

    t_day = np.array(positions).reshape(-1, 1)         # shape (24,1)
    y_day_actual = df.loc[idx_range, 'pv'].values     # actual PV values for the day

    # c) GP prediction: mean + standard deviation
    y_day_pred, y_std = gp.predict(t_day, return_std=True)
    #    (y_std is the per‐hour 1σ for that forecast)

    # d) Compute daily metrics: MAE and RMSE for those 24 points
    mae_day  = mean_absolute_error(y_day_actual, y_day_pred)
    rmse_day = np.sqrt(mean_squared_error(y_day_actual, y_day_pred))

    results.append({
        'date' : date_str,
        'MAE'  : mae_day,
        'RMSE': rmse_day
    })

    # e) Optionally, plot that day's actual vs. predicted + ±1.96σ
    plt.figure(figsize=(8, 3))
    plt.plot(idx_range, y_day_actual, 'k-', label='Actual PV')
    plt.plot(idx_range, y_day_pred,   'r-', label='GP Mean')
    upper = y_day_pred + 1.96*y_std
    lower = y_day_pred - 1.96*y_std
    plt.fill_between(idx_range, lower, upper, color='r', alpha=0.2, label='95% CI')
    plt.title(f"GP Forecast on {date_str}")
    plt.xlabel('Time (Hourly)')
    plt.ylabel('PV Output (kW)')
    plt.xticks(rotation=45)
    plt.legend(fontsize=9)
    plt.tight_layout()
    plt.show()